# Imports

In [3]:
import pandas as pd
import heapq
from collections import defaultdict
from operator import itemgetter
from lightfm.data import Dataset
from lightfm import lightfm
from lightfm.evaluation import auc_score
import numpy as np

/Users/diego/miniforge3/envs/tf/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [4]:
olist_reviews = pd.read_csv("../data/processed/olist_order_reviews_reduced.csv")
olist_products = pd.read_csv("../data/processed/olist_products_dataset_english.csv")
olist_orders = pd.read_csv("../data/processed/olist_orders_reduced.csv")
olist_customers = pd.read_csv("../data/processed/olist_customers_dataset.csv")
olist_orders_items = pd.read_csv("../data/processed/olist_order_items_reduced.csv")

In [5]:
olist_products.head()

,Unnamed: 0,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares


In [6]:
olist_reviews.head()

,Unnamed: 0,review_id,order_id,review_score
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


In [7]:
olist_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purchase_month,purchase_time
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,10,night
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,7,night
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,8,morning
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,11,evening
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2,afternoon


In [8]:
olist_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


# Data preparation

In [9]:
def prepare_product_features_df(olist_reviews_df, olist_products_df, olist_order_items_df):
    merged_review_order =  olist_reviews.merge(right=olist_orders_items, how="inner", on="order_id" )
    merged_review_order_products = merged_review_order.merge(right=olist_products, how="inner", on="product_id")
    product_df = merged_review_order_products[["review_score", "price", "product_category_name_english", "product_name_lenght", "product_description_lenght", "product_weight_g", "product_id"]]
    return product_df
product_df = prepare_product_features_df(olist_reviews, olist_products, olist_orders_items)

In [10]:
def prepare_customer_features_df(olist_customers, olist_orders, olist_reviews):
    olist_customers_orders = olist_customers.merge(right=olist_orders, how='inner', on='customer_id')
    customers_orders = olist_customers_orders.merge(right=olist_orders_items, how='inner', on='order_id')
    customers = customers_orders.merge(right=olist_reviews, how='inner', on="order_id")
    return customers
customers = prepare_customer_features_df(olist_customers, olist_orders, olist_reviews)

In [11]:
def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping 

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    features_name : List
        List of feature columns name avaiable in dataframe. 
        
    Returns
    -------
    List of all features for mapping 
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features

In [12]:
customer_features_list = generate_feature_list(customers, ['customer_city'])
product_features_list = generate_feature_list(product_df, ['price', "product_category_name_english", "product_name_lenght", "product_description_lenght","product_weight_g"])

In [13]:
dataset = Dataset()
dataset.fit(items=set(product_df['product_id']), users=set(customers['customer_unique_id']), item_features=product_features_list, user_features=customer_features_list)

In [14]:
customers['weight'] = customers['review_score'] - 3
customer_product_interaction = list(zip(customers.customer_unique_id, customers.product_id, customers.weight))

In [15]:
interactions, weights = dataset.build_interactions(data=customer_product_interaction)

In [16]:
def func(x):
    return ','.join(x.map(str))

In [18]:
def create_features (dataframe, features_name, id_col):
    features = dataframe[features_name].apply(func, axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col], features))
    return features

In [19]:
customer_fea = create_features(customers, ["customer_city"], "customer_unique_id")
customer_features = dataset.build_user_features(data=customer_fea)

In [20]:
product_fea = create_features(product_df, ['price', "product_category_name_english", "product_name_lenght", "product_description_lenght","product_weight_g"], 'product_id')
product_features = dataset.build_item_features(data=product_fea)

# Model training

In [26]:
model = lightfm.LightFM(no_components=150, learning_rate=0.05, loss='warp', random_state=2022)
model.fit(interactions=interactions, item_features=product_features, user_features=customer_features, sample_weight=weights, epochs=50, verbose=True, num_threads=16)

Epoch: 100%|██████████| 50/50 [00:52<00:00,  1.06s/it]


In [27]:
def calculate_auc_score(lightfm_model, interactions_matrix, 
                        question_features, professional_features): 
    """
    Measure the ROC AUC metric for a model. 
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model 
        A fitted lightfm model 
    interactions_matrix : 
        A lightfm interactions matrix 
    question_features, professional_features: 
        Lightfm features 
        
    Returns
    -------
    String containing AUC score 
    """
    score = auc_score( 
        lightfm_model, interactions_matrix, 
        item_features=question_features, 
        user_features=professional_features, 
        num_threads=16).mean()
    return score

In [39]:
calculate_auc_score(model, interactions, product_features, customer_features)

0.7436696

# Top N recommendation

In [163]:
def get_top_recommendations(customer_unique_ids, k, model, product_df, customers_df):
    """
    Takes in a list of customer unique IDs and returns top K recommendations for that user.

    customer_unique_ids: the unique customer id. 

    k: the number of recommendations to return
    """
    
    customer_to_internal = dataset.mapping()[0]
    internal_to_customers =  {v: k for k, v in dataset.mapping()[0].items()}
    product_to_internal = dataset.mapping()[2]
    total_recommendations = []
    for customer_id in customer_unique_ids:
        internal_id = customer_to_internal[customer_id]
        score = model.predict(internal_id, np.arange(product_df['product_id'].nunique()))
        df_scores = pd.DataFrame()
        df_scores['scores']= score
        df_scores['product_id'] = product_to_internal.keys()
        df_scores = df_scores.sort_values("scores", ascending=False).head(k)
        recs = {"customer_id" : customer_id}
        curr_recs = []
        for i in range(k):
            row = product_df.loc[product_df['product_id'] == df_scores.iloc[i, 1]]
            curr = {"product_id" : row.iloc[0,6], "recommendation_score" : df_scores.iloc[i, 0], "product_category_name_english" : row.iloc[0,2], "price": row.iloc[0,1]}
            curr_recs.append(curr)
        recs['Recommendations'] = curr_recs
        total_recommendations.append(recs)
    return total_recommendations
        
get_top_recommendations(["c64ede6d0ae8901b1b6fb03528c1b7e6", '060e732b5b29e8181a18229c7b0b2b5e'], 5, model, product_df, customers)

[{'customer_id': 'c64ede6d0ae8901b1b6fb03528c1b7e6',
  'Recommendations': [{'product_id': '8eae309a711208a0e5e41e110cddb1ee',
    'recommendation_score': 13.522766,
    'product_category_name_english': 'drinks',
    'price': 23.9},
   {'product_id': 'ee3d532c8a438679776d222e997606b3',
    'recommendation_score': 12.071263,
    'product_category_name_english': 'computers_accessories',
    'price': 100.0},
   {'product_id': '928e52a9ad53a294fdcc91bcf59d1751',
    'recommendation_score': 12.027552,
    'product_category_name_english': 'housewares',
    'price': 130.0},
   {'product_id': '26320e4682c6a0f297d07f3c74155666',
    'recommendation_score': 11.156043,
    'product_category_name_english': 'office_furniture',
    'price': 179.99},
   {'product_id': '270516a3f41dc035aa87d220228f844c',
    'recommendation_score': 9.539123,
    'product_category_name_english': 'health_beauty',
    'price': 1.2}]},
 {'customer_id': '060e732b5b29e8181a18229c7b0b2b5e',
  'Recommendations': [{'product_id'